In [1]:
import uuid
from googleads import adwords
import sys

In [2]:
from csv import reader
import pandas as pd
import numpy as np

In [3]:
AUTH_FILE_PATH = '/home/tim_su/ai_optimizer/opt/ai_optimizer/googleads.yaml'
client = adwords.AdWordsClient.LoadFromStorage(AUTH_FILE_PATH)
#client_2 = ad_manager.AdManagerClient.LoadFromStorage('/home/tim_su/ai_optimizer/opt/ai_optimizer/googleads.yaml')

In [ ]:
# ID of your customer here
# CUSTOMER_SERVICE_ID = '5922380045'
CUSTOMER_SERVICE_ID = '3637290511'
# Load customer account access
client.SetClientCustomerId(CUSTOMER_SERVICE_ID)

In [ ]:
# Display which field is able to query for certain report type
REPORT_TYPE = 'CAMPAIGN_PERFORMANCE_REPORT'

def get_report_fields(client, report_type):
    # Initialize appropriate service.
    report_definition_service = client.GetService(
        'ReportDefinitionService', version='v201809')

    # Get report fields.
    fields = report_definition_service.getReportFields(report_type)

    # Display results.
    print('Report type "%s" contains the following fields:' % report_type)
    for field in fields:
        print(' - %s(%s)' % (field['fieldName'], field['fieldType']))
        if 'enumValues' in field:
            print('  := [%s]' % ', '.join(field['enumValues']))

if __name__ == '__main__':
    # Initialize client object.
    adwords_client = adwords.AdWordsClient.LoadFromStorage(AUTH_FILE_PATH)
    get_report_fields(adwords_client, REPORT_TYPE)

In [ ]:
campaign_id = 1747836661

In [ ]:
CAMPAIGN_FIELDS = ['ExternalCustomerId','CampaignId','AdvertisingChannelType', 'CampaignStatus',
                   'BiddingStrategyType','Amount','StartDate','EndDate','Cost',
                   'AverageCost','Impressions', 'Clicks','Conversions',
                   'AverageCpc','CostPerConversion']
class DatePreset:
    today = 'TODAY'
    yesterday = 'YESTERDAY'
    lifetime = 'ALL_TIME'

def get_campaign_insights(client, campaign_id=None, date_preset=None):
    client.SetClientCustomerId(CUSTOMER_SERVICE_ID)
    report_downloader = client.GetReportDownloader(version='v201809')
    if date_preset is None:
        date_preset = 'ALL_TIME'
    else:
        date_preset = date_preset
    # Create report definition.
    report = {
        'reportName': 'CAMPAIGN_PERFORMANCE_REPORT',
#         'dateRangeType': 'CUSTOM_DATE',
        'dateRangeType': date_preset,
        'reportType': 'CAMPAIGN_PERFORMANCE_REPORT',
        'downloadFormat': 'CSV',
        'selector': {
            'fields': CAMPAIGN_FIELDS,
#             'dateRange': {'min': '20190301','max': '20190401'},
            'predicates': [
                {
                    'field': 'CampaignId',
                    'operator': 'EQUALS',
                    'values':[campaign_id]
                }
            ]
        }
    }
    # Print out the report as a string
    csv = report_downloader.DownloadReportAsString(  
        report, skip_report_header=True, skip_column_header=True,   
        skip_report_summary=True, include_zero_impressions=True,client_customer_id=CUSTOMER_SERVICE_ID)
    csv_list = csv.split('\n')[:-1]
    df = pd.DataFrame()
    for lil_csv in csv_list:
        df_temp = pd.DataFrame(
            data=np.array(lil_csv.split(',')).reshape(1,len(CAMPAIGN_FIELDS)),
            columns=CAMPAIGN_FIELDS
        )
        df = df.append(df_temp)
    df[df.columns.difference(['CampaignStatus', 'AdvertisingChannelType', 'BiddingStrategyType', 'StartDate', 'EndDate'])] = df[df.columns.difference(
        ['CampaignStatus', 'AdvertisingChannelType', 'BiddingStrategyType', 'StartDate', 'EndDate'])].apply(pd.to_numeric, errors='coerce')
    df[df.columns.difference(['ExternalCustomerId', 'CampaignId', 'AdvertisingChannelType', 'CampaignStatus', 'BiddingStrategyType', 'StartDate', 'EndDate', 'Impressions', 'Clicks', 'Conversions'])] = df[df.columns.difference(
        ['ExternalCustomerId', 'CampaignId', 'CampaignStatus', 'AdvertisingChannelType', 'BiddingStrategyType', 'StartDate', 'EndDate', 'Impressions', 'Clicks', 'Conversions'])].div(1000000)
    df[['StartDate','EndDate']] = df[['StartDate','EndDate']].apply( pd.to_datetime, errors='coerce' )
    return df

if __name__ == '__main__':
    # Initialize client object.
    client = adwords.AdWordsClient.LoadFromStorage(AUTH_FILE_PATH)
    campaign_insights = get_campaign_insights(client, campaign_id=campaign_id)


In [ ]:
campaign_insights.info()
campaign_insights

In [ ]:
ADGROUP_FIELDS = ['ExternalCustomerId','CampaignId', 'AdGroupType', 'AdGroupId', 'AdGroupStatus',
                  'CpmBid','CpvBid', 'CpcBid', 'TargetCpa', 'BiddingStrategyType','Cost',
                   'AverageCost','Impressions', 'Clicks','Conversions',
                   'AverageCpc','CostPerConversion']
# ADGROUP_FIELDS = ['ExternalCustomerId','CampaignId', 'AdGroupType', 'AdGroupId','ConversionCategoryName']
def get_adgroup_insights(client, campaign_id=None, adgroup_id=None, date_preset=None):
    client.SetClientCustomerId(CUSTOMER_SERVICE_ID)
    report_downloader = client.GetReportDownloader(version='v201809')
    if date_preset is None:
        date_preset = 'ALL_TIME'
    else:
        date_preset = date_preset
    # Create report definition.
    if campaign_id is not None and adgroup_id is None:
        operand = {
                    'field': 'CampaignId',
                    'operator': 'EQUALS',
                    'values':[campaign_id]
        }
    elif adgroup_id is not None:
        operand = {
                    'field': 'AdGroupId',
                    'operator': 'EQUALS',
                    'values':[adgroup_id]
        }
    else:
        print('get_adgroup_insights: Missing arguments campaign_id or adgroup_id.' )
        operand = None
        return
    report = {
        'reportName': 'ADGROUP_PERFORMANCE_REPORT',
#         'dateRangeType': 'CUSTOM_DATE',
        'dateRangeType': date_preset,
        'reportType': 'ADGROUP_PERFORMANCE_REPORT',
        'downloadFormat': 'CSV',
        'selector': {
            'fields': ADGROUP_FIELDS,
#             'dateRange': {'min': '20190301','max': '20190401'},
            'predicates': [
                operand
            ]
        }
    }
    # Print out the report as a string
    csv = report_downloader.DownloadReportAsString(  
        report, skip_report_header=True, skip_column_header=True,   
        skip_report_summary=True, include_zero_impressions=True,client_customer_id=CUSTOMER_SERVICE_ID)
    csv_list = csv.split('\n')[:-1]
    df = pd.DataFrame()
    for lil_csv in csv_list:
        df_temp = pd.DataFrame(
            data=np.array(lil_csv.split(',')).reshape(1,len(ADGROUP_FIELDS)),
            columns=ADGROUP_FIELDS
        )
        df = df.append(df_temp)
    df[df.columns.difference(['AdGroupType', 'AdGroupStatus', 'BiddingStrategyType'])] = df[df.columns.difference(
        ['AdGroupType', 'AdGroupStatus', 'BiddingStrategyType'])].apply(pd.to_numeric, errors='coerce')
    df[df.columns.difference( ['ExternalCustomerId', 'CampaignId', 'AdGroupType', 'AdGroupId', 'AdGroupStatus', 'BiddingStrategyType', 'Conversions'] )] = df[df.columns.difference(
        ['ExternalCustomerId', 'CampaignId', 'AdGroupType', 'AdGroupId', 'AdGroupStatus', 'BiddingStrategyType', 'Conversions'])].div(1000000)

    return df.reset_index(drop=True)

if __name__ == '__main__':
    # Initialize client object.
    client = adwords.AdWordsClient.LoadFromStorage(AUTH_FILE_PATH)
#     adgroup_insights = get_adgroup_insights(client)
    adgroup_insights = get_adgroup_insights(client, campaign_id=campaign_id)

In [ ]:
adgroup_insights.info()
adgroup_insights

In [ ]:
AGE_RANGE_FIELDS = ['ExternalCustomerId','CampaignId', 'AdGroupId', 'Id', 'AdGroupStatus',
                  'CpmBid', 'CpcBid', 'BiddingStrategyType','Cost',
                   'AverageCost','Impressions', 'Clicks','Conversions',
                   'AverageCpc','CostPerConversion']
# AGE_RANGE_FIELDS = ['ExternalCustomerId','CampaignId', 'AdGroupType', 'AdGroupId','ConversionCategoryName']
def get_age_range_insights(client, campaign_id=None, adgroup_id=None, date_preset=None):
    client.SetClientCustomerId(CUSTOMER_SERVICE_ID)
    report_downloader = client.GetReportDownloader(version='v201809')
    if date_preset is None:
        date_preset = 'ALL_TIME'
    else:
        date_preset = date_preset
    # Create report definition.
    if campaign_id is not None and adgroup_id is None:
        operand = {
                    'field': 'CampaignId',
                    'operator': 'EQUALS',
                    'values':[campaign_id]
        }
    elif adgroup_id is not None:
        operand = {
                    'field': 'AdGroupId',
                    'operator': 'EQUALS',
                    'values':[adgroup_id]
        }
    else:
        print('get_age_range_insights: Missing arguments campaign_id or adgroup_id.' )
        operand = None
        return
    report = {
        'reportName': 'AGE_RANGE_PERFORMANCE_REPORT',
#         'dateRangeType': 'CUSTOM_DATE',
        'dateRangeType': date_preset,
        'reportType': 'AGE_RANGE_PERFORMANCE_REPORT',
        'downloadFormat': 'CSV',
        'selector': {
            'fields': AGE_RANGE_FIELDS,
#             'dateRange': {'min': '20190301','max': '20190401'},
            'predicates': [
                operand
            ]
        }
    }
    # Print out the report as a string
    csv = report_downloader.DownloadReportAsString(  
        report, skip_report_header=True, skip_column_header=True,   
        skip_report_summary=True, include_zero_impressions=True,client_customer_id=CUSTOMER_SERVICE_ID)
    csv_list = csv.split('\n')[:-1]
    df = pd.DataFrame()
    for lil_csv in csv_list:
        df_temp = pd.DataFrame(
            data=np.array(lil_csv.split(',')).reshape(1,len(AGE_RANGE_FIELDS)),
            columns=AGE_RANGE_FIELDS
        )
        df = df.append(df_temp)
#     df[df.columns.difference(['AdGroupType', 'AdGroupStatus', 'BiddingStrategyType'])] = df[df.columns.difference(
#         ['AdGroupType', 'AdGroupStatus', 'BiddingStrategyType'])].apply(pd.to_numeric, errors='coerce')
#     df[df.columns.difference( ['ExternalCustomerId', 'CampaignId', 'AdGroupType', 'AdGroupId', 'AdGroupStatus', 'BiddingStrategyType', 'Conversions'] )] = df[df.columns.difference(
#         ['ExternalCustomerId', 'CampaignId', 'AdGroupType', 'AdGroupId', 'AdGroupStatus', 'BiddingStrategyType', 'Conversions'])].div(1000000)

    return df.reset_index(drop=True)

if __name__ == '__main__':
    # Initialize client object.
    client = adwords.AdWordsClient.LoadFromStorage(AUTH_FILE_PATH)
#     adgroup_insights = get_age_range_insights(client)
    adgroup_insights = get_age_range_insights(client, campaign_id=1747836670, adgroup_id=68524904775)

In [ ]:
AUDIENCE_FIELDS = ['ExternalCustomerId','CampaignId', 'AdGroupId', 'Id', 'AdGroupStatus', 'UserListName',
                  'CpmBid', 'CpcBid', 'BiddingStrategyType','Cost',
                   'AverageCost','Impressions', 'Clicks','Conversions',
                   'AverageCpc','CostPerConversion']
# AUDIENCE_FIELDS = ['ExternalCustomerId','CampaignId', 'AdGroupType', 'AdGroupId','ConversionCategoryName']
def get_audience_insights(client, campaign_id=None, adgroup_id=None, date_preset=None):
    client.SetClientCustomerId(CUSTOMER_SERVICE_ID)
    report_downloader = client.GetReportDownloader(version='v201809')
    if date_preset is None:
        date_preset = 'ALL_TIME'
    else:
        date_preset = date_preset
    # Create report definition.
    if campaign_id is not None and adgroup_id is None:
        operand = {
                    'field': 'CampaignId',
                    'operator': 'EQUALS',
                    'values':[campaign_id]
        }
    elif adgroup_id is not None:
        operand = {
                    'field': 'AdGroupId',
                    'operator': 'EQUALS',
                    'values':[adgroup_id]
        }
    else:
        print('get_audience_insights: Missing arguments campaign_id or adgroup_id.' )
        operand = None
        return
    report = {
        'reportName': 'AUDIENCE_PERFORMANCE_REPORT',
#         'dateRangeType': 'CUSTOM_DATE',
        'dateRangeType': date_preset,
        'reportType': 'AUDIENCE_PERFORMANCE_REPORT',
        'downloadFormat': 'CSV',
        'selector': {
            'fields': AUDIENCE_FIELDS,
#             'dateRange': {'min': '20190301','max': '20190401'},
            'predicates': [
                operand
            ]
        }
    }
    # Print out the report as a string
    csv = report_downloader.DownloadReportAsString(  
        report, skip_report_header=True, skip_column_header=True,   
        skip_report_summary=True, include_zero_impressions=True,client_customer_id=CUSTOMER_SERVICE_ID)
    print(csv)
    csv_list = csv.split('\n')[:-1]
    df = pd.DataFrame()
    for lil_csv in csv_list:
        df_temp = pd.DataFrame(
            data=np.array(lil_csv.split(',')).reshape(1,len(AUDIENCE_FIELDS)),
            columns=AUDIENCE_FIELDS
        )
        df = df.append(df_temp)
#     df[df.columns.difference(['AdGroupType', 'AdGroupStatus', 'BiddingStrategyType'])] = df[df.columns.difference(
#         ['AdGroupType', 'AdGroupStatus', 'BiddingStrategyType'])].apply(pd.to_numeric, errors='coerce')
#     df[df.columns.difference( ['ExternalCustomerId', 'CampaignId', 'AdGroupType', 'AdGroupId', 'AdGroupStatus', 'BiddingStrategyType', 'Conversions'] )] = df[df.columns.difference(
#         ['ExternalCustomerId', 'CampaignId', 'AdGroupType', 'AdGroupId', 'AdGroupStatus', 'BiddingStrategyType', 'Conversions'])].div(1000000)

    return df.reset_index(drop=True)

if __name__ == '__main__':
    # Initialize client object.
    client = adwords.AdWordsClient.LoadFromStorage(AUTH_FILE_PATH)
#     adgroup_insights = get_audience_insights(client)
    adgroup_insights = get_audience_insights(client, campaign_id=1747836670, adgroup_id=68524904775)

In [ ]:
adgroup_insights

In [ ]:
CUSTOMER_SERVICE_ID = 3637290511

In [ ]:
adgroup_id = 1747836661

In [ ]:
client = adwords.AdWordsClient.LoadFromStorage(AUTH_FILE_PATH)
ad_group_service = client.GetService('AdGroupService', version='v201809')

# Construct operations and update an ad group.
operations = [{
    'operator': 'SET',
    'operand': {
        'id': ad_group_id,
        'status': 'ACTIVE'
    }
}]

if bid_micro_amount:
    operations[0]['operand']['biddingStrategyConfiguration'] = {
      'bids': [{
          'xsi_type': 'CpcBid',
          'bid': {
              'microAmount': bid_micro_amount,
          }
      }]
    }

ad_groups = ad_group_service.mutate(operations)

In [6]:
from googleads import adwords
CUSTOMER_SERVICE_ID = '3637290511'
client.SetClientCustomerId(CUSTOMER_SERVICE_ID)
AD_GROUP_ID = 68524904055
CPC_BID_MICRO_AMOUNT = '4000000'


def update_adgroup_bid(client, ad_group_id, bid_micro_amount=None):
    # Initialize appropriate service.
    ad_group_service = client.GetService('AdGroupService', version='v201809')
    
    # Construct operations and update an ad group.
    operations = [{
        'operator': 'SET',
        'operand': {
            'id': ad_group_id,
            'status': 'ENABLED'
        }
    }]
  
    if bid_micro_amount:
        operations[0]['operand']['biddingStrategyConfiguration'] = {
            'bids': [{
                'xsi_type': 'CpcBid',
                'bid': {
                    'microAmount': bid_micro_amount,
                }
            }]
        }
  
    ad_groups = ad_group_service.mutate(operations)
    return ad_groups

if __name__ == '__main__':
    # Initialize client object.
    adwords_client = adwords.AdWordsClient.LoadFromStorage(AUTH_FILE_PATH)
    adwords_client.SetClientCustomerId(CUSTOMER_SERVICE_ID)
    update_adgroup_bid(adwords_client, AD_GROUP_ID, CPC_BID_MICRO_AMOUNT)

{
    'ListReturnValue.Type': 'AdGroupReturnValue',
    'value': [
        {
            'id': 68524904055,
            'campaignId': 1747836661,
            'campaignName': '華友旅行_4月_CPC$8_3/28-4/27_BT',
            'name': 'BT-愛家人士',
            'status': 'ENABLED',
            'settings': [
                {
                    'Setting.Type': 'TargetingSetting',
                    'details': [
                        {
                            'criterionTypeGroup': 'USER_INTEREST_AND_LIST',
                            'targetAll': False
                        },
                        {
                            'criterionTypeGroup': 'VERTICAL',
                            'targetAll': False
                        },
                        {
                            'criterionTypeGroup': 'GENDER',
                            'targetAll': False
                        },
                        {
                            'criterionTypeGroup': 'AGE_RANGE',
            

In [ ]:
CUSTOMER_SERVICE_ID = '3207327801'
client.SetClientCustomerId(CUSTOMER_SERVICE_ID)
CAMPAIGN_ID = 1745022368
AD_GROUP_ID = 74005199208
CPC_BID_MICRO_AMOUNT = '8000000'
adwords_client = adwords.AdWordsClient.LoadFromStorage(AUTH_FILE_PATH)
adwords_client.SetClientCustomerId(CUSTOMER_SERVICE_ID)
update_adgroup_bid(adwords_client, AD_GROUP_ID, CPC_BID_MICRO_AMOUNT)

In [ ]:
# CUSTOMER_SERVICE_ID = '3637290511'
client.SetClientCustomerId(CUSTOMER_SERVICE_ID)
ad_group_service = client.GetService('AdGroupService', version='v201809')

# Construct operations and update an ad group.

operations = [{
    'operator': 'SET',
    'operand': {
        'id': 68524904775,
        'status': 'ENABLED'
    }
}]
# ad_groups = ad_group_service.get(selector)
ad_groups = ad_group_service.mutate(operations)

In [ ]:
ad_groups

In [7]:
CUSTOMER_SERVICE_ID = '3637290511'
client.SetClientCustomerId(CUSTOMER_SERVICE_ID)
campaign_service = client.GetService('CampaignService', version='v201809')

# Construct operations and update an ad group.

operations = [{
    'operator': 'SET',
    'operand': {
        'id': 1747836670,
        'status': 'ENABLED'
    }
}]
# ad_groups = ad_group_service.get(selector)
campaigns = campaign_service.mutate(operations)

In [8]:
campaigns

{
    'ListReturnValue.Type': 'CampaignReturnValue',
    'value': [
        {
            'id': 1747836670,
            'campaignGroupId': None,
            'name': '華友旅行_4月_CPC$8_3/28-4/27_PT',
            'status': 'ENABLED',
            'servingStatus': 'SERVING',
            'startDate': '20190328',
            'endDate': '20190427',
            'budget': {
                'budgetId': 1806201010,
                'name': '華友旅行_4月_CPC$8_3/28-4/27_PT',
                'amount': {
                    'ComparableValue.Type': 'Money',
                    'microAmount': 100000000
                },
                'deliveryMethod': 'STANDARD',
                'referenceCount': None,
                'isExplicitlyShared': False,
                'status': 'ENABLED'
            },
            'conversionOptimizerEligibility': {
                'eligible': False,
                'rejectionReasons': [
                    'NOT_ENOUGH_CONVERSIONS'
                ]
            },
            'adS

In [ ]:
def main(client):
    # Initialize appropriate service.
    constant_data_service = client.GetService(
        'ConstantDataService', version='v201809')
  
    # Get all ageranges.
    ageranges = constant_data_service.getAgeRangeCriterion()
    print(ageranges)
    # Display results.
    for agerange in ageranges:
        print ('agerange with name "%s" and ID "%s" was found.'
               % (agerange['ageRangeType'], agerange['id']))
  
    # Get all carriers.
    carriers = constant_data_service.getCarrierCriterion()
  
    # Display results.
    for carrier in carriers:
        print ('Carrier with name "%s", ID "%s", and country code "%s" was '
               'found.' % (
                   carrier['name'], carrier['id'],
                   getattr(carrier, 'countryCode', 'N/A')))


if __name__ == '__main__':
    # Initialize client object.
    adwords_client = adwords.AdWordsClient.LoadFromStorage(AUTH_FILE_PATH)
  
    main(adwords_client)